In [2]:
import json, requests
import Levenshtein as lev
import unicodedata
import pandas as pd

In [3]:

ID = "aaf20ac1"
key = "7bd88c15c64569da8de3c414ace8d236"

baseURL = "http://api.yummly.com/v1/api/recipes?_app_id="+ID+"&_app_key="+key+"&q=@&maxResult=50"
baseURLGet = "http://api.yummly.com/v1/api/recipe/@?_app_id="+ID+"&_app_key="+key

In [4]:
def convertName(name):
    converted = unicodedata.normalize('NFKD', name).encode('ascii','ignore')
    return converted.lower()

def bestMatch(query, response):
    tmpQuery = query.lower()
    levDistances = [lev.distance(convertName(match["recipeName"]), tmpQuery) for match in response["matches"]]
    return response["matches"][levDistances.index(min(levDistances))]

def getRecipe(candidateID):
    url = baseURLGet.replace("@", candidateID)
    response = json.loads(requests.get(url).text)
    return response

def buildJson(original, searchResponse, recipeResponse):
    jsonBuilder = {}

    jsonBuilder["RecipeURL"] = recipeResponse["source"]["sourceRecipeUrl"]
    jsonBuilder["CleanIngredients"] = searchResponse["ingredients"]
    jsonBuilder["RawIngredients"] = recipeResponse["ingredientLines"]
    jsonBuilder["RecipeName"] = searchResponse["recipeName"]
    jsonBuilder["FoodName"] = original
    return jsonBuilder
    

In [6]:
def searchFromList():
    responses = []
    notFound = []
    foodList = pd.read_csv("food101.csv", sep=',')
    for query in foodList['FoodName']:
        print "query: ", query
        params = query.lower().replace(" ", "+")
        url = baseURL.replace("@",params)
        response = json.loads(requests.get(url).text)
        if not response["matches"]:
            "Hello no matches :3"
            notFound.append(query)
    
        else:
            candidate = bestMatch(query, response)
            candidateID = candidate["id"]
            recipeResponse = getRecipe(candidateID)
            builtJson = buildJson(query, candidate, recipeResponse)
            responses.append(builtJson)
            
    data = json.dumps(responses)
    with open('recipesData.json', 'w') as f:
        json.dump(responses, f)
    with open('notFound.json', 'w') as f:
        json.dump(notFound, f)
    
    return responses, notFound
    
responses, notFound = searchFromList()

#print responses
#print notFound

query:  Apple pie
query:  Baby back ribs
query:  Baklava
query:  Beef carpaccio
query:  Beef tartare
query:  Beet salad
query:  Beignets
query:  Bibimbap
query:  Bread pudding
query:  Breakfast burrito
query:  Bruschetta
query:  Caesar salad
query:  Cannoli
query:  Caprese salad
query:  Carrot cake
query:  Ceviche
query:  Cheesecake
query:  Cheese plate
query:  Chicken curry
query:  Chicken quesadilla
query:  Chicken wings
query:  Chocolate cake
query:  Chocolate mousse
query:  Churros
query:  Clam chowder
query:  Club sandwich
query:  Crab cakes
query:  Creme brulee
query:  Croque madame
query:  Cupcakes
query:  Deviled eggs
query:  Donuts
query:  Dumplings
query:  Edamame
query:  Eggs benedict
query:  Escargots
query:  Falafel
query:  Filet mignon
query:  Fish and chips
query:  Foie gras
query:  French fries
query:  French onion soup
query:  French toast
query:  Fried calamari
query:  Fried rice
query:  Frozen yogurt
query:  Garlic bread
query:  Gnocchi
query:  Greek salad
query:  Gr

[u'flour', u'pizza doughs', u'canned tomatoes', u'low-fat mozzarella cheese', u'basil leaves', u'salt', u'olive oil']

[u'canola oil', u'finely chopped onion', u'chopped celery', u'diced tomatoes', u'water', u'brown sugar', u'salt', u'dried basil', u'dried oregano', u'ground pepper']

[u'yellow onion', u'beef broth', u'baguette', u'bay leaf', u'gruyere cheese', u'unsalted butter', u'all-purpose flour', u'extra-virgin olive oil', u'balsamic vinegar', u'worcestershire sauce', u'kosher salt', u'freshly ground pepper', u'chives']

[u'sun-dried tomatoes', u'chicken breasts', u'garlic cloves', u'salt', u'paprika', u'half & half', u'asiago', u'penne pasta', u'fresh spinach']

[u'lean ground beef', u'whole wheat spaghetti', u'marinara sauce', u'chopped onion', u'carrots', u'chopped celery', u'minced garlic', u'oregano', u'extra-virgin olive oil', u'sea salt', u'black pepper']

